[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adan-rs/amd/blob/main/notebooks/09_Escalamiento_multidimensional.ipynb)

# Escalamiento multidimensional

*¿Para qué se utiliza?*
El escalamiento multidimensional (EMD) es una técnica que permite construir una representación espacial o geométrica de objetos (productos, marcas, servicios) a partir de las percepciones o preferencias de los encuestados. Su uso más común en investigación de mercados es el mapa perceptual, que permite visualizar el posicionamiento relativo de marcas según cómo son percibidas o preferidas por los consumidores.

*¿Qué variables considera?*
- Juicios de semejanza o disimilitud (percepción directa): evaluaciones de qué tan similares o diferentes son los objetos.
- Calificaciones por atributos (percepción derivada): evaluación de marcas en atributos específicos (precio, calidad, etc.).
- Preferencias: ordenamientos o comparaciones de marcas según el agrado o intención de compra.
Los datos pueden obtenerse mediante escalas Likert o de diferencial semántico, comparaciones por pares, ordenamientos simples.

*¿Cómo funciona?*
El EMD utiliza los datos de semejanza o preferencia para estimar distancias euclidianas entre los objetos, y luego los ubica en un espacio de 2 o más dimensiones tal que esas distancias sean lo más fieles posible a los juicios recogidos. Cuanto más cercanos estén dos objetos en el mapa, más similares son percibidos por los encuestados.

Tipos de escalamiento:
    Métrico: usa distancias o calificaciones numéricas (intervalo o razón).
    No métrico: usa ordenamientos o rangos, preserva solo la estructura de orden.

*Evaluación del ajuste*
- Stress-I: mide el error de representación espacial. Valores menores a 0.10 indican un buen ajuste.
- R² (R cuadrada): mide la proporción de varianza explicada por la configuración. Se recomiendan valores mayores a 0.60.
- Otros indicadores como dispersión contada o congruencia de Tucker también evalúan la calidad de la solución.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Para este ejemplo utilizaremos una matriz de correlaciones para calcular la matriz de distancias

In [ ]:
from sklearn.manifold import MDS

In [ ]:
#df = pd.read_excel('../data/enigh2020.xlsx')
df = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/enigh2020.xlsx')

In [ ]:
variables = ["ing_cor", "gasto_mon", "edad_jefe", "tot_integ"]

In [ ]:
def realizar_mds(df, variables):
    '''
    Realiza escalamiento multidimensional con base en correlaciones
    '''
    # Validar variables
    for var in variables:
       if var not in df.columns:
           raise ValueError(f"Variable {var} no encontrada en el DataFrame")
       if not np.issubdtype(df[var].dtype, np.number):
           raise ValueError(f"Variable {var} debe ser numérica")    
    
    # Calcular matriz de correlaciones y distancias
    matriz_corr = df[variables].corr()
    distancias = 1-np.abs(matriz_corr)
    
    # Aplicar modelo
    mds = MDS(n_components=2, dissimilarity='precomputed', normalized_stress='auto', random_state=42)
    mds_resultados = mds.fit_transform(distancias)

    # Convertir resultados a dataframe
    mds_df = pd.DataFrame(mds_resultados, columns=['Dimension 1', 'Dimension 2'])
    mds_df['Etiqueta']=variables

    #Visualización
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=mds_df, x='Dimension 1', y='Dimension 2')
    for i in range(len(mds_df)):
        plt.text(mds_df['Dimension 1'][i], mds_df['Dimension 2'][i], 
             mds_df['Etiqueta'][i])
    plt.show()
    
    return mds_df

In [ ]:
resultado = realizar_mds(df, variables)

Ejemplo de reporte de resultados:
>Se aplicó escalamiento multidimensional (EMD) no métrico a una matriz de disimilitudes obtenida mediante juicios pareados sobre cinco marcas de supermercados. La configuración espacial de dos dimensiones explicó el 91% de la varianza, y el valor de Stress-I fue 0.072, indicando un ajuste adecuado. En el mapa perceptual resultante, Marca A y Marca E fueron percibidas como similares entre sí, mientras que Marca D fue percibida como distinta a todas las demás. Se solicitó a los encuestados que interpretaran las dimensiones, identificándolas como "precio percibido" y "variedad de productos".
